In [ ]:
#CÉLULA 1 (DATASET 89k - 16x16)
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt

#Constantes
IMAGE_SIZE = 16
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)
DATA_PATH = '../input/pixel-art/sprites.npy'

#Carregar e Pré-processar
print("Carregando o dataset .npy...")
X_train_raw = np.load(DATA_PATH)

print(f"Dataset bruto carregado. Shape: {X_train_raw.shape}")

def normalize_image(img_array):
    return (img_array.astype(np.float32) - 127.5) / 127.5

X_train = normalize_image(X_train_raw)

#Verificação de Sanidade
print(f"\nDataset processado com sucesso.")
print(f"Formato (Shape): {X_train.shape}") #Deve ser (89000, 16, 16, 3)
print(f"Valor Mínimo: {X_train.min():.2f}") #Deve ser -1.0
print(f"Valor Máximo: {X_train.max():.2f}") #Deve ser 1.0

print("\nExibindo uma amostra (desnormalizada):")
sample_image_to_show = (X_train[0] + 1) / 2.0
plt.imshow(sample_image_to_show)
plt.axis('off')
plt.show()

In [ ]:
#CÉLULA 2 (WGAN-GP 16x16)
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

#Constantes de Arquitetura
NOISE_DIM = 100
BATCH_SIZE = 64

#Gerador (16x16)
def build_generator():
    """Converte ruído (100) -> imagem (16x16x3)"""
    model = Sequential(name="Generator_16x16")
    
    #Semente 4x4
    model.add(layers.Dense(4 * 4 * 256, input_dim=NOISE_DIM))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Reshape((4, 4, 256)))
    
    #4x4 -> 8x8
    model.add(layers.UpSampling2D())
    model.add(layers.Conv2D(128, kernel_size=3, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))

    #8x8 -> 16x16
    model.add(layers.UpSampling2D())
    model.add(layers.Conv2D(128, kernel_size=3, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))

    #Camada de Saída (16x16x3)
    model.add(layers.Conv2D(3, kernel_size=3, padding='same', activation='tanh'))
    
    print("--- Arquitetura do Gerador (16x16) ---")
    model.summary()
    return model

#Crítico (16x16)
def build_critic():
    """Converte imagem (16x16x3) -> score (1)"""
    model = Sequential(name="Critic_16x16")
    
    #16x16 -> 8x8
    model.add(layers.Conv2D(128, kernel_size=3, strides=2, padding='same', input_shape=IMAGE_SHAPE))
    model.add(layers.LeakyReLU(alpha=0.2))

    #8x8 -> 4x4
    model.add(layers.Conv2D(256, kernel_size=3, strides=2, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))

    #Camada de Análise (4x4)
    model.add(layers.Conv2D(512, kernel_size=3, padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))

    #Saída
    model.add(layers.Flatten())
    model.add(layers.Dense(1)) 
    
    print("\n--- Arquitetura do Crítico (16x16) ---")
    model.summary()
    return model

#Otimizadores
#Com 89k de dados, o Crítico é "gênio". Não precisamos mais do 1e-6.
#Voltamos para a taxa padrão (1e-4) que "explodia" antes.
critic_optimizer = Adam(learning_rate=0.0001, beta_1=0.0, beta_2=0.9)
generator_optimizer = Adam(learning_rate=0.0001, beta_1=0.0, beta_2=0.9)

#Criar os Modelos
generator = build_generator()
critic = build_critic()

In [ ]:
#CÉLULA 3 (VERSÃO WGAN-GP LOOP - 16x16)
import numpy as np
import matplotlib.pyplot as plt
import time
import tensorflow as tf

#Constantes de Treino
EPOCHS = 500
D_STEPS_PER_G_STEP = 5 
GP_WEIGHT = 10.0   
SAVE_INTERVAL = 20

#Função de Salvar
def save_plot(generator, epoch, noise_dim):
    """
    Gera uma grade 4x4 de imagens do Gerador e a salva em disco.
    """
    noise = tf.random.normal([16, noise_dim])
    generated_images = generator(noise, training=False)
    #Desnormaliza de [-1, 1] para [0, 1] para visualização
    generated_images = (generated_images + 1) / 2.0
    
    fig, axs = plt.subplots(4, 4, figsize=(8, 8))
    for i in range(16):
        axs[i//4, i%4].imshow(generated_images[i])
        axs[i//4, i%4].axis('off')
    fig.savefig(f"/kaggle/working/gan_generated_epoch_{epoch}.png")
    plt.close()

#Funções de Perda WGAN-GP
def critic_loss(real_output, fake_output):
    """Perda do Crítico: maximizar (real_score - fake_score)"""
    real_loss = tf.reduce_mean(real_output)
    fake_loss = tf.reduce_mean(fake_output)
    return fake_loss - real_loss

def generator_loss(fake_output):
    """Perda do Gerador: maximizar (fake_score)"""
    return -tf.reduce_mean(fake_output)

#Função da Penalidade de Gradiente (O "GP")
def gradient_penalty(real_images, fake_images):
    """Calcula a Penalidade de Gradiente (GP) para o WGAN-GP"""
    #Amostra aleatória de pontos na linha entre imagens reais e falsas
    alpha = tf.random.uniform([BATCH_SIZE, 1, 1, 1], 0.0, 1.0)
    #Cria as imagens interpoladas
    interpolated = real_images + alpha * (fake_images - real_images)

    with tf.GradientTape() as gp_tape:
        gp_tape.watch(interpolated)
        #Obtém a predição (score) do Crítico para essas imagens
        pred = critic(interpolated, training=True)
    
    #Calcula o gradiente do score em relação às imagens interpoladas
    grads = gp_tape.gradient(pred, [interpolated])[0]
    #Calcula a norma (magnitude) de cada gradiente
    norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
    #Calcula a penalidade: a média do desvio quadrático da norma de 1
    gp = tf.reduce_mean((norm - 1.0) ** 2)
    return gp

#O Loop de Treino Customizado (com tf.function)
#@tf.function compila a função para performance máxima
@tf.function
def train_step_critic(real_images):
    """Executa um único passo de treino para o Crítico."""
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])
    
    with tf.GradientTape() as tape:
        fake_images = generator(noise, training=True)
        
        real_output = critic(real_images, training=True)
        fake_output = critic(fake_images, training=True)
        
        c_loss = critic_loss(real_output, fake_output)
        gp = gradient_penalty(real_images, fake_images)
        
        #Perda total = Perda Wasserstein + Penalidade de Gradiente
        total_critic_loss = c_loss + gp * GP_WEIGHT
        
    critic_gradients = tape.gradient(total_critic_loss, critic.trainable_variables)
    critic_optimizer.apply_gradients(zip(critic_gradients, critic.trainable_variables))
    
    return total_critic_loss

@tf.function
def train_step_generator():
    """Executa um único passo de treino para o Gerador."""
    noise = tf.random.normal([BATCH_SIZE, NOISE_DIM])
    
    with tf.GradientTape() as tape:
        fake_images = generator(noise, training=True)
        fake_output = critic(fake_images, training=True)
        g_loss = generator_loss(fake_output)
        
    generator_gradients = tape.gradient(g_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(generator_gradients, generator.trainable_variables))
    
    return g_loss

#O Loop de Épocas
def train_gan(dataset, epochs, d_steps):
    """O loop de treino principal que orquestra as épocas e batches."""
    #Cria o dataset em lotes
    batched_dataset = tf.data.Dataset.from_tensor_slices(dataset).shuffle(len(dataset)).batch(BATCH_SIZE, drop_remainder=True)
    
    print("Iniciando o treinamento (WGAN-GP)...")
    start_time = time.time()
    
    for epoch in range(epochs):
        epoch_start_time = time.time()
        c_loss_epoch = []
        g_loss_epoch = []
        
        for i, image_batch in enumerate(batched_dataset):
            
            #Sempre treina o Crítico
            c_loss = train_step_critic(image_batch)
            c_loss_epoch.append(c_loss)
            
            #Treina o Gerador apenas a cada 'd_steps' (5)
            if (i + 1) % d_steps == 0:
                g_loss = train_step_generator()
                g_loss_epoch.append(g_loss)
                
        #Fim da Época
        c_loss_mean = tf.reduce_mean(c_loss_epoch).numpy()
        g_loss_mean = tf.reduce_mean(g_loss_epoch).numpy() if g_loss_epoch else -1.0
        
        epoch_time = time.time() - epoch_start_time
        print(f"Época {epoch+1}/{epochs} | D Loss (Crítico): {c_loss_mean:.4f} | G Loss (Gerador): {g_loss_mean:.4f} | Tempo: {epoch_time:.2f}s")
        
        #Salva imagens de amostra em intervalos definidos
        if (epoch + 1) % SAVE_INTERVAL == 0 or epoch == 0:
            save_plot(generator, epoch + 1, NOISE_DIM)
            print(f" Imagens de exemplo salvas para a época {epoch+1}")
            
    total_time = time.time() - start_time
    print(f"\nTreinamento concluído em {total_time/60:.2f} minutos.")

#Iniciar o treinamento
train_gan(X_train, EPOCHS, D_STEPS_PER_G_STEP)